## 从atlas判断是否取用


In [20]:
import numpy as np
import numba as nb
atlas = np.load("dataset/patient1/npy/atlas.npy")
x = 240
y = 240
z = 240
atlas = atlas[x:x+128, y:y+128, z:z+128, :]

In [21]:
def if_count(atlas, ratio):
    is_organ = 0
    for i in range(atlas.shape[0]):
        for j in range(atlas.shape[1]):
            for k in range(atlas.shape[2]):
                if atlas[i, j, k, 0] != 0:
                    is_organ += 1
    if is_organ > ratio * atlas.size:
        return True
    else:
        return False

@nb.jit
def if_count_nb(atlas, ratio):
    is_organ = 0
    for i in range(atlas.shape[0]):
        for j in range(atlas.shape[1]):
            for k in range(atlas.shape[2]):
                if atlas[i, j, k, 0] != 0:
                    is_organ += 1
    if is_organ > ratio * atlas.size:
        return True
    else:
        return False

def if_count_np(atlas, ratio):
    n_count = np.sum(atlas > 0)
    if n_count/atlas.size > ratio:
        return True
    else:
        return False

In [22]:
t1 = %timeit -o if_count(atlas, 0.5)
t2 = %timeit -o if_count_np(atlas, 0.5)
if_count_nb(atlas, 0.5)
t3 = %timeit -o if_count_nb(atlas, 0.5)

2.13 s ± 31.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.04 ms ± 34.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.21 ms ± 20.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
print("NumPy sum比循环快%.2f倍"%(t1.average/t2.average))
print("Numba jit比循环快%.2f倍"%(t1.average/t3.average))
print("简单的Numba jit与NumPy sum效果接近, 但略慢%.2f"%(t3.average/t2.average))

NumPy sum比循环快703.37倍
Numba jit比循环快665.30倍
简单的Numba jit与NumPy sum效果接近, 但略慢1.06


## 对整个人体进行遍历

In [31]:
import numpy as np
import numba as nb

def create_patch(atlas, size=128, step=16, ratio=0.5):
    # 计算每个维度可取的个数
    n_i = np.floor((atlas.shape[0]-size)/step) + 1
    n_j = np.floor((atlas.shape[1]-size)/step) + 1
    n_k = np.floor((atlas.shape[2]-size)/step) + 1

    # 记录个数
    count_index = []
    # 对所有的起始点进行遍历
    for i in np.arange(0, step*n_i, step, dtype=np.uint16):
        for j in np.arange(0, step*n_j, step, dtype=np.uint16):
            for k in np.arange(0, step*n_k, step, dtype=np.uint16):
                atlas_patch = atlas[i:i+size, j:j+size, k:k+size, :]

                if np.sum(atlas_patch > 0)/atlas_patch.size > ratio:
                    count_index.append([i, j, k])
    return count_index

@nb.jit
def create_patch_jit(atlas, size=128, step=16, ratio=0.5):
    # 计算每个维度可取的个数
    n_i = np.floor((atlas.shape[0]-size)/step) + 1
    n_j = np.floor((atlas.shape[1]-size)/step) + 1
    n_k = np.floor((atlas.shape[2]-size)/step) + 1

    # 记录个数
    count_index = []
    # 对所有的起始点进行遍历
    for i in np.arange(0, step*n_i, step, dtype=np.uint16):
        for j in np.arange(0, step*n_j, step, dtype=np.uint16):
            for k in np.arange(0, step*n_k, step, dtype=np.uint16):

                atlas_patch = atlas[i:i+size, j:j+size, k:k+size, :]
                if np.sum(atlas_patch > 0)/atlas_patch.size > ratio:
                    count_index.append([i, j, k])
    return count_index


atlas = np.load("dataset/patient1/npy/atlas.npy")

In [26]:
import time

t1 = time.time()
index_loop = create_patch(atlas)
t2 = time.time()
print("简单循环:%.2fs"%(t2-t1))

t3 = time.time()
index_jit = create_patch_jit(atlas)
t4 = time.time()
print("简单循环+jit:%.2fs"%(t4-t3))

简单循环:39.16s
简单循环+jit:110.08s


In [102]:
@nb.jit(parallel=True)
def create_patch_parallel(atlas, size=128, step=16, ratio=0.5):
    # 计算每个维度可取的个数
    n_i = int(np.floor((atlas.shape[0]-size)/step) + 1)
    n_j = int(np.floor((atlas.shape[1]-size)/step) + 1)
    n_k = int(np.floor((atlas.shape[2]-size)/step) + 1)
    # 记录个数
    count_index = []
    # 对所有的起始点进行遍历
    for i in range(n_i):
        i *= step
        for j in range(n_j):
            j *= step
            for k in range(n_k):
                k *= step

                atlas_patch = atlas[i:i+size, j:j+size, k:k+size, :]
                if np.sum(atlas_patch > 0)/atlas_patch.size > ratio:
                    count_index.append([i, j, k])
    return count_index

atlas = np.load("dataset/patient1/npy/atlas.npy")
t3 = time.time()
index_parallel = create_patch_parallel(atlas)
t4 = time.time()
print("并行循环:%.2fs"%(t4-t3))

并行循环:7.09s


In [99]:
print(len(index_parallel))
print(index_parallel==index_loop)

122
True


In [103]:
@nb.jit(parallel=True)
def create_patch_parallel(atlas, size=128, step=16, ratio=0.5):
    # 计算每个维度可取的个数
    n_i = int(np.floor((atlas.shape[0]-size)/step) + 1)
    n_j = int(np.floor((atlas.shape[1]-size)/step) + 1)
    n_k = int(np.floor((atlas.shape[2]-size)/step) + 1)
    # 记录个数
    count_index = []
    # 对所有的起始点进行遍历
    for i in np.arange(0, step*n_i, step, dtype=np.uint16):
        for j in np.arange(0, step*n_j, step, dtype=np.uint16):
            for k in np.arange(0, step*n_k, step, dtype=np.uint16):

                atlas_patch = atlas[i:i+size, j:j+size, k:k+size, :]
                if np.sum(atlas_patch > 0)/atlas_patch.size > ratio:
                    count_index.append([i, j, k])
    return count_index

atlas = np.load("dataset/patient1/npy/atlas.npy")
t3 = time.time()
index_parallel = create_patch_parallel(atlas)
t4 = time.time()
print("并行循环:%.2fs"%(t4-t3))

<ipython-input-103-1a860e704489>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "create_patch_parallel" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "<ipython-input-103-1a860e704489>", line 10:
def create_patch_parallel(atlas, size=128, step=16, ratio=0.5):
    <source elided>
    # 对所有的起始点进行遍历
    for i in np.arange(0, step*n_i, step, dtype=np.uint16):
    ^

  @nb.jit(parallel=True)
D:\MySoftware\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "create_patch_parallel" was compiled in object mode without forceobj=True, but has lifted loops.

File "<ipython-input-103-1a860e704489>", line 4:
def create_patch_parallel(atlas, size=128, step=16, ratio=0.5):
    <source elided>
    # 计算每个维度可取的个数
    n_i = int(np.floor((atlas.shape[0]-size)/step) + 1)
    ^

  warnings.warn(errors.NumbaWarning(warn_msg,
D:\MySoftware\anaconda3\lib

并行循环:40.36s


In [104]:
@nb.jit
def create_patch_parallel(atlas, size=128, step=16, ratio=0.5):
    # 计算每个维度可取的个数
    n_i = int(np.floor((atlas.shape[0]-size)/step) + 1)
    n_j = int(np.floor((atlas.shape[1]-size)/step) + 1)
    n_k = int(np.floor((atlas.shape[2]-size)/step) + 1)
    # 记录个数
    count_index = []
    # 对所有的起始点进行遍历
    for i in range(n_i):
        i *= step
        for j in range(n_j):
            j *= step
            for k in range(n_k):
                k *= step

                atlas_patch = atlas[i:i+size, j:j+size, k:k+size, :]
                if np.sum(atlas_patch > 0)/atlas_patch.size > ratio:
                    count_index.append([i, j, k])
    return count_index

atlas = np.load("dataset/patient1/npy/atlas.npy")
t3 = time.time()
index_jit = create_patch_parallel(atlas)
t4 = time.time()
print("并行循环:%.2fs"%(t4-t3))

并行循环:113.73s


In [110]:
index = [[1,2,3],[4,5,6]]
arr = np.array(index)
for i, j, k in arr:
    print(i)
    print(j)
    print(k)
    print()

1
2
3

4
5
6



In [112]:
import os
os.path.exists("dataset")

True

In [115]:
os.path.exists("dataset/test.md")

False

In [128]:
print(os.path.exists("dataset/test/1/1"))
os.makedirs("dataset/test/1/1")
print(os.path.exists("dataset/test/1/1"))

False
True


In [124]:
os.path.dirname("dataset/test/1/1/text.txt")

'dataset/test/1/1'

In [125]:
os.path.basename("dataset/test/1/1/text.txt")

'text.txt'

In [133]:
a = np.array([[1,2,3],[4,5,6]])
len(a)

2